# 0. Импорт библиотек

In [13]:
# !pip install -r jupiter-requirements.txt

In [2]:
import pandas as pd
import requests
import os

In [3]:
# Тип вывода результатов в jupiter
from IPython.core.interactiveshell import InteractiveShell

jup_conds = ['all',                # Показывать все вызовы
             'last',               # Показывать последнюю общность вызовов (если последним будет идти цикл с вызовами, выведет все его вызовы)
             'last_expr',          # Показывать самый последний вызов
             'none',               # Ничего не показывать
             'last_expr_or_assign' # Показывать последний вызов или присваивание
            ]

InteractiveShell.ast_node_interactivity = jup_conds[0]

# 1. Проверка работоспособности API

In [4]:
def post_request(file_path):
    with open(file_path, 'rb') as data:
        request = requests.post('http://127.0.0.1:5000/read-book', files={'file': data})
        print(f'Загрузка книги. Код ответа «{request.status_code}»')
        # print(f'Данные: {request.content}')
        return request

In [5]:
base_dir = os.getcwd()

In [16]:
r = post_request(os.path.join(base_dir, 'Tests', 'Datasets', 'PE-81b_2semestr_24_01-13_04.xlsx')).json()
r

Загрузка книги. Код ответа «200»


{'file_hash': '19e390e4fd80a2b80f448e87289087af308324307b6314e84e3085c1e27383b4',
 'sheets': {'ПЕ-81б (2)': ['ПЕ-81б']}}

In [23]:
file_hash = r['file_hash']
result_types = ['json', 'excel']
for sheet_name, groups in r['sheets'].items():
    for group_name in groups:
        for result_type in result_types:
            request = requests.post(f'http://127.0.0.1:5000/parse-book/{file_hash}/{sheet_name}/{group_name}/{result_type}')
            request.content

b'[\n    {\n        "day":"\xd0\x9f\xd0\x9d",\n        "num":1,\n        "item_name":"\xd0\xa1\xd0\xb5\xd1\x82\xd0\xb5\xd0\xb2\xd0\xbe\xd0\xb5 \xd0\xbf\xd1\x80\xd0\xbe\xd0\xb3\xd1\x80\xd0\xb0\xd0\xbc\xd0\xbc\xd0\xbd\xd0\xbe\xd0\xb5 \xd0\xbe\xd0\xb1\xd0\xb5\xd1\x81\xd0\xbf\xd0\xb5\xd1\x87\xd0\xb5\xd0\xbd\xd0\xb8\xd0\xb5",\n        "teacher":"\xd1\x81\xd1\x82.\xd0\xbf\xd1\x80\xd0\xb5\xd0\xbf\xd0\xbe\xd0\xb4\xd0\xb0\xd0\xb2\xd0\xb0\xd1\x82\xd0\xb5\xd0\xbb\xd1\x8c \xd0\x9b\xd0\xb5\xd0\xb1\xd0\xb5\xd0\xb4\xd0\xb5\xd0\xb2 \xd0\x93.\xd0\x9a.",\n        "type":"\xd0\x9b\xd0\x91",\n        "pdgr":"2",\n        "date_pair":"2022-01-31T00:00:00.000Z",\n        "cab":"306 \xd0\xa3\xd0\x9a\xe2\x84\x961"\n    },\n    {\n        "day":"\xd0\x9f\xd0\x9d",\n        "num":1,\n        "item_name":"\xd0\xa1\xd0\xb5\xd1\x82\xd0\xb5\xd0\xb2\xd0\xbe\xd0\xb5 \xd0\xbf\xd1\x80\xd0\xbe\xd0\xb3\xd1\x80\xd0\xb0\xd0\xbc\xd0\xbc\xd0\xbd\xd0\xbe\xd0\xb5 \xd0\xbe\xd0\xb1\xd0\xb5\xd1\x81\xd0\xbf\xd0\xb5\xd1\x87\xd0\xb5

b'<!DOCTYPE html>\n<html>\n  <head>\n    <title>TypeError: The view function did not return a valid response. The return type must be a string, dict, tuple, Response instance, or WSGI callable, but it was a Workbook.\n // Werkzeug Debugger</title>\n    <link rel="stylesheet" href="?__debugger__=yes&amp;cmd=resource&amp;f=style.css">\n    <link rel="shortcut icon"\n        href="?__debugger__=yes&amp;cmd=resource&amp;f=console.png">\n    <script src="?__debugger__=yes&amp;cmd=resource&amp;f=debugger.js"></script>\n    <script>\n      var CONSOLE_MODE = false,\n          EVALEX = true,\n          EVALEX_TRUSTED = false,\n          SECRET = "pWHSwB2A1i92D5mQdfPQ";\n    </script>\n  </head>\n  <body style="background-color: #fff">\n    <div class="debugger">\n<h1>TypeError</h1>\n<div class="detail">\n  <p class="errormsg">TypeError: The view function did not return a valid response. The return type must be a string, dict, tuple, Response instance, or WSGI callable, but it was a Workbook.\n

In [21]:
r['sheets'].keys()

dict_keys(['ПЕ-81б (2)'])

In [10]:
files = [
         os.path.join(base_dir, 'Tests', 'Datasets', 'PE-81b_2semestr_24_01-13_04.xlsx'),
         os.path.join(base_dir, 'Tests', 'Datasets', 'MITE-91_2019-2020_II_semestr.xls'),
         os.path.join(base_dir, 'test_parser.ipynb'),
         os.path.join(base_dir, 'Design', 'cr_evolve.gif'),
        ]

for file_path in files:
    r = post_request(file_path)
    if r.status_code != 200:
        print(r.content.decode('utf8'))
    else:
        r.json()
        file_hash = r.json()['file_hash']
        # f = [True, True, True, True]
        for sheet_name, groups in r.json()['sheets'].items():
            for group_name in groups:
                # request = requests.post(f'http://127.0.0.1:5000/parse-book?file_hash={file_hash}&sheet_name={sheet_name}&group_name={group_name}')
                # request = requests.post(f'http://127.0.0.1:5000/parse-book/{file_hash}/{sheet_name}/{group_name}?f1={f[0]}&f2={f[1]}&f3={f[2]}&f4={f[3]}')
                request = requests.post(f'http://127.0.0.1:5000/parse-book/{file_hash}/{sheet_name}/{group_name}')
                print(f'Парсинг. Код ответа: {request.status_code}')
                try:
                    json_data = request.json()
                    json_data[:3]
                except:
                    print('Сервак вернул не JSON')
    print()

Загрузка книги. Код ответа «200»


{'file_hash': '19e390e4fd80a2b80f448e87289087af308324307b6314e84e3085c1e27383b4',
 'sheets': {'ПЕ-81б (2)': ['ПЕ-81б']}}

Парсинг. Код ответа: 200


[{'day': 'ПН',
  'num': 1,
  'item_name': 'Сетевое программное обеспечение',
  'teacher': 'ст.преподаватель Лебедев Г.К.',
  'type': 'ЛБ',
  'pdgr': '2',
  'date_pair': '2022-01-31T00:00:00.000Z',
  'cab': '306 УК№1'},
 {'day': 'ПН',
  'num': 1,
  'item_name': 'Сетевое программное обеспечение',
  'teacher': 'ст.преподаватель Лебедев Г.К.',
  'type': 'ЛБ',
  'pdgr': '2',
  'date_pair': '2022-02-07T00:00:00.000Z',
  'cab': '306 УК№1'},
 {'day': 'ПН',
  'num': 1,
  'item_name': 'Сетевое программное обеспечение',
  'teacher': 'ст.преподаватель Лебедев Г.К.',
  'type': 'ЛБ',
  'pdgr': '2',
  'date_pair': '2022-02-14T00:00:00.000Z',
  'cab': '306 УК№1'}]


Загрузка книги. Код ответа «200»


{'file_hash': '354828d30a445f828c48657f7566703e727832eeec1cc6977147d6df851eec31',
 'sheets': {'МИТЕ-91': ['МИТЕ-91']}}

Парсинг. Код ответа: 200


[{'day': 'ПН',
  'num': 1,
  'item_name': 'Современные технологии в программировании',
  'teacher': 'доцент Обвинцев О.А.',
  'type': 'лекция',
  'pdgr': 'общ',
  'date_pair': '2020-02-10T00:00:00.000Z',
  'cab': '310 УК№1'},
 {'day': 'ПН',
  'num': 1,
  'item_name': 'Современные технологии в программировании',
  'teacher': 'доцент Обвинцев О.А.',
  'type': 'лекция',
  'pdgr': 'общ',
  'date_pair': '2020-03-23T00:00:00.000Z',
  'cab': '310 УК№1'},
 {'day': 'ПН',
  'num': 1,
  'item_name': 'Современные технологии в программировании',
  'teacher': 'доцент Обвинцев О.А.',
  'type': 'ЛБ',
  'pdgr': 'общ',
  'date_pair': '2020-03-30T00:00:00.000Z',
  'cab': '310 УК№1'}]


Загрузка книги. Код ответа «415»
Файл должен иметь расширение «.xls» или «.xlsx»

Загрузка книги. Код ответа «413»
<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>413 Request Entity Too Large</title>
<h1>Request Entity Too Large</h1>
<p>The data value transmitted exceeds the capacity limit.</p>




In [11]:
pd.DataFrame(json_data).head()

,day,num,item_name,teacher,type,pdgr,date_pair,cab
0,ПН,1,Современные технологии в программировании,доцент Обвинцев О.А.,лекция,общ,2020-02-10T00:00:00.000Z,310 УК№1
1,ПН,1,Современные технологии в программировании,доцент Обвинцев О.А.,лекция,общ,2020-03-23T00:00:00.000Z,310 УК№1
2,ПН,1,Современные технологии в программировании,доцент Обвинцев О.А.,ЛБ,общ,2020-03-30T00:00:00.000Z,310 УК№1
3,ПН,2,Волоконно-оптические системы передачи,доцент Кусайкин Д.В.,лекция,общ,2020-02-10T00:00:00.000Z,101 УК№3
4,ПН,2,Волоконно-оптические системы передачи,доцент Кусайкин Д.В.,лекция,общ,2020-02-17T00:00:00.000Z,101 УК№3


In [12]:
r.headers

{'Server': 'Werkzeug/2.1.1 Python/3.9.1', 'Date': 'Mon, 25 Apr 2022 04:28:00 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '196'}

# 2. Тестирование API